## Slice the large CSV into a data cube with figures aggregated

In [19]:
# import libraries
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# read data
df = pd.read_csv('../../../data/crashes.csv')

## Loading checking

In [2]:
print(df.head())
df.shape

                                     CRASH_RECORD_ID     RD_NO  \
0  79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...  JC199149   
1  792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...  JB422857   
2  0115ade9a755e835255508463f7e9c4a9a0b47e9304238...  JF318029   
3  017040c61958d2fa977c956b2bd2d6759ef7754496dc96...  JF324552   
4  78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...  JB291672   

  CRASH_DATE_EST_I  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
0              NaN                  30         TRAFFIC SIGNAL   
1              NaN                  30            NO CONTROLS   
2              NaN                  30                UNKNOWN   
3              NaN                  30         TRAFFIC SIGNAL   
4              NaN                  30            NO CONTROLS   

       DEVICE_CONDITION WEATHER_CONDITION      LIGHTING_CONDITION  \
0  FUNCTIONING PROPERLY             CLEAR                DAYLIGHT   
1           NO CONTROLS             CLEAR                DAYLIGHT   
2    

(541604, 53)

## People involved and injured

In [6]:
# calculate the total of row count, INJURIES_TOTAL, INJURIES_FATAL, INJURIES_INCAPACITATING, INJURIES_NON_INCAPACITATING, INJURIES_REPORTED_NOT_EVIDENT, INJURIES_NO_INDICATION, INJURIES_UNKNOWN
injuries = df.groupby(['CRASH_YEAR','SIDE']).agg({'INJURIES_TOTAL':'sum','INJURIES_FATAL':'sum','INJURIES_INCAPACITATING':'sum','INJURIES_NON_INCAPACITATING':'sum','INJURIES_REPORTED_NOT_EVIDENT':'sum','INJURIES_NO_INDICATION':'sum'}).reset_index()

# convert columns other than 'CRASH_YEAR', 'SIDE' to int
injuries.iloc[:,2:] = injuries.iloc[:,2:].fillna(0).astype(int)

# select columns to be used
crash_ppl_involved = injuries[['CRASH_YEAR','SIDE','INJURIES_TOTAL','INJURIES_NO_INDICATION']]
crash_ppl_injured = injuries[['CRASH_YEAR','SIDE','INJURIES_FATAL','INJURIES_INCAPACITATING','INJURIES_NON_INCAPACITATING','INJURIES_REPORTED_NOT_EVIDENT']]

# rename columns
crash_ppl_involved.columns = ['CRASH_YEAR','SIDE','Injured','No indication of injury']
crash_ppl_injured.columns = ['CRASH_YEAR','SIDE','Fatal','Incapacitating','Non-incapacitating','Reported but not evident']

# output to JSON
crash_ppl_involved.to_json('crash_ppl_involved.json',orient='records',indent=2)
crash_ppl_injured.to_json('crash_ppl_injured.json',orient='records',indent=2)

## Time

In [8]:
#pivot CRASH_DAY_OF_WEEK and CRASH_HOUR to create a new column for each month and day
crash_day_time = df.pivot_table(index=['CRASH_YEAR','SIDE'], columns=['CRASH_DAY_OF_WEEK','CRASH_HOUR'], values='CRASH_RECORD_ID', aggfunc='count').reset_index()

#convert columns other than 'CRASH_YEAR', 'SIDE' to int
crash_day_time.iloc[:,2:] = crash_day_time.iloc[:,2:].fillna(0).astype(int)

#flatten the column to 1 layer and rename the columns of CRASH_DAY_OF_WEEK and CRASH_HOUR into a comma seperated string
crash_day_time.columns = ['CRASH_YEAR','SIDE', \
                          'Sun,0','Sun,1','Sun,2','Sun,3','Sun,4','Sun,5','Sun,6','Sun,7','Sun,8','Sun,9','Sun,10','Sun,11','Sun,12','Sun,13','Sun,14','Sun,15','Sun,16','Sun,17','Sun,18','Sun,19','Sun,20','Sun,21','Sun,22','Sun,23', \
                          'Mon,0','Mon,1','Mon,2','Mon,3','Mon,4','Mon,5','Mon,6','Mon,7','Mon,8','Mon,9','Mon,10','Mon,11','Mon,12','Mon,13','Mon,14','Mon,15','Mon,16','Mon,17','Mon,18','Mon,19','Mon,20','Mon,21','Mon,22','Mon,23', \
                          'Tue,0','Tue,1','Tue,2','Tue,3','Tue,4','Tue,5','Tue,6','Tue,7','Tue,8','Tue,9','Tue,10','Tue,11','Tue,12','Tue,13','Tue,14','Tue,15','Tue,16','Tue,17','Tue,18','Tue,19','Tue,20','Tue,21','Tue,22','Tue,23', \
                          'Wed,0','Wed,1','Wed,2','Wed,3','Wed,4','Wed,5','Wed,6','Wed,7','Wed,8','Wed,9','Wed,10','Wed,11','Wed,12','Wed,13','Wed,14','Wed,15','Wed,16','Wed,17','Wed,18','Wed,19','Wed,20','Wed,21','Wed,22','Wed,23', \
                          'Thu,0','Thu,1','Thu,2','Thu,3','Thu,4','Thu,5','Thu,6','Thu,7','Thu,8','Thu,9','Thu,10','Thu,11','Thu,12','Thu,13','Thu,14','Thu,15','Thu,16','Thu,17','Thu,18','Thu,19','Thu,20','Thu,21','Thu,22','Thu,23', \
                          'Fri,0','Fri,1','Fri,2','Fri,3','Fri,4','Fri,5','Fri,6','Fri,7','Fri,8','Fri,9','Fri,10','Fri,11','Fri,12','Fri,13','Fri,14','Fri,15','Fri,16','Fri,17','Fri,18','Fri,19','Fri,20','Fri,21','Fri,22','Fri,23', \
                          'Sat,0','Sat,1','Sat,2','Sat,3','Sat,4','Sat,5','Sat,6','Sat,7','Sat,8','Sat,9','Sat,10','Sat,11','Sat,12','Sat,13','Sat,14','Sat,15','Sat,16','Sat,17','Sat,18','Sat,19','Sat,20','Sat,21','Sat,22','Sat,23']
crash_day_time.head()

#output to JSON
crash_day_time.to_json('crash_day_time.json', orient='records',indent=2)

## Causes

In [2]:
causes = df.pivot_table(index=['CRASH_YEAR','SIDE'],columns='PRIM_CONTRIBUTORY_CAUSE',values='CRASH_RECORD_ID',aggfunc='count').reset_index()

# convert columns other than 'CRASH_YEAR', 'SIDE' to int
causes.iloc[:,2:] = causes.iloc[:,2:].fillna(0).astype(int)

# ensure the names are sorted in alphabetical order
cols = causes.columns.tolist()
# remove CRASH_YEAR and SIDE from the list
cols.remove('CRASH_YEAR')
cols.remove('SIDE')
# reorder the columns
cols.sort()
cols = ['CRASH_YEAR', 'SIDE'] + cols
causes = causes[cols]

# rename the columns
causes.columns = ['CRASH_YEAR','SIDE', \
                'Animal', \
                'Bicycle advancing legally on red light', \
                'Cell phone use other than texting', \
                'Disregarding other traffic signs', \
                'Disregarding road markings', \
                'Disregarding stop sign', \
                'Disregarding traffic signals', \
                'Disregarding yield sign', \
                'Distraction - from inside vehicle', \
                'Distraction - from outside vehicle', \
                'Distraction - other electronic device (navigation device, DVD player, etc.)', \
                'Driving on wrong side/wrong way', \
                'Driving skills/knowledge/experience', \
                'Vehicle condition', \
                'Evasive action due to animal, object, nonmotorist', \
                'Exceeding authorized speed limit', \
                'Exceeding safe speed for conditions', \
                'Failing to reduce speed to avoid crash', \
                'Failing to yield right-of-way', \
                'Following too closely', \
                'Had been drinking (not arrested)', \
                'Improper backing', \
                'Improper lane usage', \
                'Improper overtaking/passing', \
                'Improper turning/no signal', \
                'Motorcycle advancing legally on red light', \
                'Not applicable', \
                'Obstructed crosswalks', \
                'Operating vehicle in erratic, reckless, careless, negligent or aggressive manner', \
                'Passing stopped school bus', \
                'Physical condition of driver', \
                'Related to bus stop', \
                'Road construction/maintenance', \
                'Road engineering/surface/marking defects', \
                'Texting', \
                'Turning right on red', \
                'Unable to determine', \
                'Under the influence of alcohol/drugs (arrested)', \
                'Vision obscured (signs, tree limbs, buildings, etc.)', \
                'Weather']

# output to JSON
causes.to_json('crash_cause.json', orient='records',indent=2)

## Environment (Sankey)

In [28]:
# extract the environmental conditions
env = df[['CRASH_YEAR','SIDE','CRASH_RECORD_ID','LIGHTING_CONDITION','WEATHER_CONDITION','ROADWAY_SURFACE_COND']]

lighting_dict = {
  'DARKNESS': 'Darkness', 
  'DARKNESS, LIGHTED ROAD': 'Darkness', 
  'DAWN': 'Dawn', 
  'DAYLIGHT': 'Daylight', 
  'DUSK': 'Dusk', 
  'UNKNOWN': 'Unknown lighting'
}

weather_dict = {
  'BLOWING SAND, SOIL, DIRT': 'Other/Unknown weather', 
  'BLOWING SNOW': 'Rain/Snow', 
  'CLEAR': 'Clear', 
  'CLOUDY/OVERCAST': 'Cloudy', 
  'FOG/SMOKE/HAZE': 'Other/Unknown weather', 
  'FREEZING RAIN/DRIZZLE': 'Rain/Snow', 
  'OTHER': 'Other/Unknown weather', 
  'RAIN': 'Rain/Snow', 
  'SEVERE CROSS WIND GATE': 'Other/Unknown weather', 
  'SLEET/HAIL': 'Rain/Snow', 
  'SNOW': 'Rain/Snow', 
  'UNKNOWN': 'Other/Unknown weather'
}

roadway_dict = {
  'DRY': 'Dry', 
  'ICE': 'Snow/Ice', 
  'OTHER': 'Other/Unknown surface', 
  'SAND, MUD, DIRT': 'Other/Unknown surface', 
  'SNOW OR SLUSH': 'Snow/Ice', 
  'UNKNOWN': 'Other/Unknown surface', 
  'WET': 'Wet'
}

env['LIGHTING_GRP'] = env['LIGHTING_CONDITION'].map(lighting_dict)
env['WEATHER_GRP'] = env['WEATHER_CONDITION'].map(weather_dict)
env['ROADWAY_GRP'] = env['ROADWAY_SURFACE_COND'].map(roadway_dict)

# pivot table for LIGHTING_GRP and WEATHER_GRP
env_lighting_weather = env.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR', 'SIDE'], columns=['LIGHTING_GRP', 'WEATHER_GRP'], aggfunc='count', fill_value=0)

# pivot table for WEATHER_GRP and ROADWAY_GRP
env_weather_roadway = env.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR', 'SIDE'], columns=['WEATHER_GRP', 'ROADWAY_GRP'], aggfunc='count', fill_value=0)

# merge the two pivot tables
env_lighting_weather_roadway = pd.merge(env_lighting_weather, env_weather_roadway, on=['CRASH_YEAR', 'SIDE'])
env_lighting_weather_roadway.reset_index(inplace=True)
env_lighting_weather_roadway.columns = ['CRASH_YEAR','SIDE', \
                                        'Darkness,Clear', 'Darkness,Cloudy', 'Darkness,Other/Unknown weather', 'Darkness,Rain/Snow', \
                                        'Dawn,Clear', 'Dawn,Cloudy', 'Dawn,Other/Unknown weather', 'Dawn,Rain/Snow', \
                                        'Daylight,Clear', 'Daylight,Cloudy', 'Daylight,Other/Unknown weather', 'Daylight,Rain/Snow', \
                                        'Dusk,Clear', 'Dusk,Cloudy', 'Dusk,Other/Unknown weather', 'Dusk,Rain/Snow', \
                                        'Unknown lighting,Clear', 'Unknown lighting,Cloudy', 'Unknown lighting,Other/Unknown weather', 'Unknown lighting,Rain/Snow', \
                                        'Clear,Dry', 'Clear,Snow/Ice', 'Clear,Other/Unknown surface', 'Clear,Wet', \
                                        'Cloudy,Dry', 'Cloudy,Snow/Ice', 'Cloudy,Other/Unknown surface', 'Cloudy,Wet', \
                                        'Other/Unknown weather,Dry', 'Other/Unknown weather,Snow/Ice', 'Other/Unknown weather,Other/Unknown surface', 'Other/Unknown weather,Wet', \
                                        'Rain/Snow,Dry', 'Rain/Snow,Snow/Ice', 'Rain/Snow,Other/Unknown surface', 'Rain/Snow,Wet']
env_lighting_weather_roadway.head()

# output to JSON
env_lighting_weather_roadway.to_json('crash_env.json', orient='records',indent=2)

/var/folders/nd/2yz24zyn6rv1nphhbkb_mt480000gn/T/ipykernel_40041/1494314885.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  env['LIGHTING_GRP'] = env['LIGHTING_CONDITION'].map(lighting_dict)
/var/folders/nd/2yz24zyn6rv1nphhbkb_mt480000gn/T/ipykernel_40041/1494314885.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  env['WEATHER_GRP'] = env['WEATHER_CONDITION'].map(weather_dict)
/var/folders/nd/2yz24zyn6rv1nphhbkb_mt480000gn/T/ipykernel_40041/1494314885.py:40: SettingWithCopyWarning: 
A value is tryi

## Enviornment (Severity)

In [26]:
weather = df[['CRASH_YEAR','SIDE','CRASH_RECORD_ID','WEATHER_CONDITION','MOST_SEVERE_INJURY']]

weather_dict = {
  'BLOWING SAND, SOIL, DIRT': 'Other/Unknown weather', 
  'BLOWING SNOW': 'Rain/Snow', 
  'CLEAR': 'Clear', 
  'CLOUDY/OVERCAST': 'Cloudy', 
  'FOG/SMOKE/HAZE': 'Other/Unknown weather', 
  'FREEZING RAIN/DRIZZLE': 'Rain/Snow', 
  'OTHER': 'Other/Unknown weather', 
  'RAIN': 'Rain/Snow', 
  'SEVERE CROSS WIND GATE': 'Other/Unknown weather', 
  'SLEET/HAIL': 'Rain/Snow', 
  'SNOW': 'Rain/Snow', 
  'UNKNOWN': 'Other/Unknown weather'
}

injury_dict = {
  'FATAL': 'Severe injury',
  'INCAPACITATING INJURY': 'Severe injury',
  'NO INDICATION OF INJURY': 'No injury',
  'NONINCAPACITATING INJURY': 'Moderate injury',
  'REPORTED, NOT EVIDENT': 'No injury'
}

weather['WEATHER_GRP'] = weather['WEATHER_CONDITION'].map(weather_dict)
weather['INJURY_GRP'] = weather['MOST_SEVERE_INJURY'].map(injury_dict)

weather_inju = weather.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR','SIDE'], columns=['WEATHER_GRP','INJURY_GRP'], aggfunc='count', fill_value=0)
weather_inju.columns = ['Clear,Moderate injury', 'Clear,No injury', 'Clear,Severe injury', \
                        'Cloudy,Moderate injury', 'Cloudy,No injury', 'Cloudy,Severe injury', \
                        'Other/Unknown weather,Moderate injury', 'Other/Unknown weather,No injury', 'Other/Unknown weather,Severe injury', \
                        'Rain/Snow,Moderate injury', 'Rain/Snow,No injury', 'Rain/Snow,Severe injury']
weather_inju.head()

Clear,Moderate injury  Clear,No injury  \
CRASH_YEAR SIDE                                                         
2018       Central                               651             9429   
           Far North Side                        805            10725   
           Far Southeast Side                    599             6212   
           Far Southwest Side                    457             5107   
           North Side                            649             7828   

                               Clear,Severe injury  Cloudy,Moderate injury  \
CRASH_YEAR SIDE                                                              
2018       Central                             149                      25   
           Far North Side                      193                      45   
           Far Southeast Side                  142                      35   
           Far Southwest Side                  117                      19   
           North Side                          145                      25   

                               Cloudy,No injury  Cloudy,Severe injury  \
CRASH_YEAR SIDE                                                         
2018       Central                          278                     8   
           Far North Side                   417                     7   
           Far Southeast Side               267                     5   
           Far Southwest Side               280                     6   
           North Side                       321                     2   

                               Other/Unknown weather,Moderate injury  \
CRASH_YEAR SIDE                                                        
2018       Central                                                17   
           Far North Side                                         28   
           Far Southeast Side                                     20   
           Far Southwest Side                                     17   
           North Side                                             22   

                               Other/Unknown weather,No injury  \
CRASH_YEAR SIDE                                                  
2018       Central                                         424   
           Far North Side                                  802   
           Far Southeast Side                              428   
           Far Southwest Side                              335   
           North Side                                      602   

                               Other/Unknown weather,Severe injury  \
CRASH_YEAR SIDE                                                      
2018       Central                                               1   
           Far North Side                                        3   
           Far Southeast Side                                    0   
           Far Southwest Side                                    2   
           North Side                                            8   

                               Rain/Snow,Moderate injury  Rain/Snow,No injury  \
CRASH_YEAR SIDE                                                                 
2018       Central                                   135                 1348   
           Far North Side                            155                 1857   
           Far Southeast Side                        120                 1045   
           Far Southwest Side                         89                  855   
           North Side                                115                 1231   

                               Rain/Snow,Severe injury  
CRASH_YEAR SIDE                                         
2018       Central                                  40  
           Far North Side                           45  
           Far Southeast Side                       26  
           Far Southwest Side                       14  
           North Side                               31

In [28]:
# extract the environmental conditions
lighting = df[['CRASH_YEAR','SIDE','CRASH_RECORD_ID','LIGHTING_CONDITION','MOST_SEVERE_INJURY']]

lighting_dict = {
  'DARKNESS': 'Darkness', 
  'DARKNESS, LIGHTED ROAD': 'Darkness', 
  'DAWN': 'Dawn', 
  'DAYLIGHT': 'Daylight', 
  'DUSK': 'Dusk', 
  'UNKNOWN': 'Unknown lighting'
}

injury_dict = {
  'FATAL': 'Severe injury',
  'INCAPACITATING INJURY': 'Severe injury',
  'NO INDICATION OF INJURY': 'No injury',
  'NONINCAPACITATING INJURY': 'Moderate injury',
  'REPORTED, NOT EVIDENT': 'No injury'
}

lighting['LIGHTING_GRP'] = lighting['LIGHTING_CONDITION'].map(lighting_dict)
lighting['INJURY_GRP'] = lighting['MOST_SEVERE_INJURY'].map(injury_dict)

lighting_inju = lighting.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR','SIDE'], columns=['LIGHTING_GRP','INJURY_GRP'], aggfunc='count', fill_value=0)
lighting_inju.columns = ['Darkness,Moderate injury', 'Darkness,No injury', 'Darkness,Severe injury', \
                          'Dawn,Moderate injury', 'Dawn,No injury', 'Dawn,Severe injury', \
                          'Daylight,Moderate injury', 'Daylight,No injury', 'Daylight,Severe injury', \
                          'Dusk,Moderate injury', 'Dusk,No injury', 'Dusk,Severe injury', \
                          'Unknown lighting,Moderate injury', 'Unknown lighting,No injury', 'Unknown lighting,Severe injury']
lighting_inju.head()

Darkness,Moderate injury  Darkness,No injury  \
CRASH_YEAR SIDE                                                               
2018       Central                                  288                3094   
           Far North Side                           286                3102   
           Far Southeast Side                       251                2141   
           Far Southwest Side                       195                1645   
           North Side                               254                2670   

                               Darkness,Severe injury  Dawn,Moderate injury  \
CRASH_YEAR SIDE                                                               
2018       Central                                 74                    18   
           Far North Side                          76                    21   
           Far Southeast Side                      66                    14   
           Far Southwest Side                      42                    17   
           North Side                              68                    20   

                               Dawn,No injury  Dawn,Severe injury  \
CRASH_YEAR SIDE                                                     
2018       Central                        206                   5   
           Far North Side                 240                   3   
           Far Southeast Side             166                   6   
           Far Southwest Side             112                   4   
           North Side                     126                   1   

                               Daylight,Moderate injury  Daylight,No injury  \
CRASH_YEAR SIDE                                                               
2018       Central                                  497                7616   
           Far North Side                           690                9493   
           Far Southeast Side                       471                5080   
           Far Southwest Side                       342                4381   
           North Side                               503                6428   

                               Daylight,Severe injury  Dusk,Moderate injury  \
CRASH_YEAR SIDE                                                               
2018       Central                                110                    24   
           Far North Side                         158                    30   
           Far Southeast Side                      98                    34   
           Far Southwest Side                      87                    19   
           North Side                             111                    25   

                               Dusk,No injury  Dusk,Severe injury  \
CRASH_YEAR SIDE                                                     
2018       Central                        291                   7   
           Far North Side                 352                  11   
           Far Southeast Side             272                   3   
           Far Southwest Side             218                   6   
           North Side                     304                   4   

                               Unknown lighting,Moderate injury  \
CRASH_YEAR SIDE                                                   
2018       Central                                            1   
           Far North Side                                     6   
           Far Southeast Side                                 4   
           Far Southwest Side                                 9   
           North Side                                         9   

                               Unknown lighting,No injury  \
CRASH_YEAR SIDE                                             
2018       Central                                    272   
           Far North Side                             614   
           Far Southeast Side                         293   
           Far Southwest Side                         221   
           

In [30]:
# extract the environmental conditions
roadway = df[['CRASH_YEAR','SIDE','CRASH_RECORD_ID','ROADWAY_SURFACE_COND','MOST_SEVERE_INJURY']]

roadway_dict = {
  'DRY': 'Dry', 
  'ICE': 'Snow/Ice', 
  'OTHER': 'Other/Unknown surface', 
  'SAND, MUD, DIRT': 'Other/Unknown surface', 
  'SNOW OR SLUSH': 'Snow/Ice', 
  'UNKNOWN': 'Other/Unknown surface', 
  'WET': 'Wet'
}

injury_dict = {
  'FATAL': 'Severe injury',
  'INCAPACITATING INJURY': 'Severe injury',
  'NO INDICATION OF INJURY': 'No injury',
  'NONINCAPACITATING INJURY': 'Moderate injury',
  'REPORTED, NOT EVIDENT': 'No injury'
}

roadway['ROADWAY_GRP'] = roadway['ROADWAY_SURFACE_COND'].map(roadway_dict)
roadway['INJURY_GRP'] = roadway['MOST_SEVERE_INJURY'].map(injury_dict)

roadway_inju = roadway.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR','SIDE'], columns=['ROADWAY_GRP','INJURY_GRP'], aggfunc='count', fill_value=0)
roadway_inju.columns = ['Dry,Moderate injury', 'Dry,No injury', 'Dry,Severe injury', \
                        'Other/Unknown surface,Moderate injury', 'Other/Unknown surface,No injury', 'Other/Unknown surface,Severe injury', \
                        'Snow/Ice,Moderate injury', 'Snow/Ice,No injury', 'Snow/Ice,Severe injury', \
                        'Wet,Moderate injury', 'Wet,No injury', 'Wet,Severe injury']
roadway_inju.head()

Dry,Moderate injury  Dry,No injury  \
CRASH_YEAR SIDE                                                     
2018       Central                             616           8977   
           Far North Side                      773          10015   
           Far Southeast Side                  577           5841   
           Far Southwest Side                  427           4929   
           North Side                          600           7157   

                               Dry,Severe injury  \
CRASH_YEAR SIDE                                    
2018       Central                           140   
           Far North Side                    180   
           Far Southeast Side                137   
           Far Southwest Side                113   
           North Side                        136   

                               Other/Unknown surface,Moderate injury  \
CRASH_YEAR SIDE                                                        
2018       Central                                                22   
           Far North Side                                         34   
           Far Southeast Side                                     30   
           Far Southwest Side                                     21   
           North Side                                             49   

                               Other/Unknown surface,No injury  \
CRASH_YEAR SIDE                                                  
2018       Central                                         578   
           Far North Side                                 1109   
           Far Southeast Side                              552   
           Far Southwest Side                              422   
           North Side                                     1026   

                               Other/Unknown surface,Severe injury  \
CRASH_YEAR SIDE                                                      
2018       Central                                               3   
           Far North Side                                       10   
           Far Southeast Side                                    2   
           Far Southwest Side                                    4   
           North Side                                            8   

                               Snow/Ice,Moderate injury  Snow/Ice,No injury  \
CRASH_YEAR SIDE                                                               
2018       Central                                   26                 319   
           Far North Side                            41                 693   
           Far Southeast Side                        33                 433   
           Far Southwest Side                        24                 289   
           North Side                                23                 417   

                               Snow/Ice,Severe injury  Wet,Moderate injury  \
CRASH_YEAR SIDE                                                              
2018       Central                                  7                  164   
           Far North Side                           6                  185   
           Far Southeast Side                       1                  134   
           Far Southwest Side                       0                  110   
           North Side                               9                  139   

                               Wet,No injury  Wet,Severe injury  
CRASH_YEAR SIDE                                                  
2018       Central                      1605                 48  
           Far North Side               1984                 52  
           Far Southeast Side           1126                 33  
           Far Southwest Side            937                 22  
           North Side                   1382                 33

In [24]:
# merge lighting_inju, weather_inju and roadway_inju
env = pd.merge(lighting_inju, weather_inju, on=['CRASH_YEAR', 'SIDE'])
env = pd.merge(env, roadway_inju, on=['CRASH_YEAR', 'SIDE'])

env.head(50)

LIGHTING_GRP                         Darkness                          \
INJURY_GRP                    Moderate injury No injury Severe injury   
CRASH_YEAR SIDE                                                         
2018       Central                        288      3094            74   
           Far North Side                 286      3102            76   
           Far Southeast Side             251      2141            66   
           Far Southwest Side             195      1645            42   
           North Side                     254      2670            68   
           Northwest Side                 221      2445            59   
           South Side                     369      2533           100   
           Southwest Side                 380      3562           112   
           West Side                      709      5897           190   
2019       Central                        294      3069            81   
           Far North Side                 253      3101            72   
           Far Southeast Side             269      2147            68   
           Far Southwest Side             157      1616            43   
           North Side                     277      2409            62   
           Northwest Side                 242      2417            65   
           South Side                     352      2635            72   
           Southwest Side                 367      3565            88   
           West Side                      741      5731           185   
2020       Central                        191      1722            50   
           Far North Side                 211      2273            76   
           Far Southeast Side             316      2244            90   
           Far Southwest Side             193      1620            56   
           North Side                     190      1477            47   
           Northwest Side                 179      1838            48   
           South Side                     367      2698            82   
           Southwest Side                 412      3448           122   
           West Side                      744      5335           227   
2021       Central                        288      2325            85   
           Far North Side                 243      2532            65   
           Far Southeast Side             371      2502           104   
           Far Southwest Side             229      1686            51   
           North Side                     231      1918            54   
           Northwest Side                 221      2096            61   
           South Side                     472      3009           116   
           Southwest Side                 484      4016           121   
           West Side                      773      5881           209   
2022       Central                        299      2568            80   
           Far North Side                 278      2598            60   
           Far Southeast Side             300      2123            78   
           Far Southwest Side             179      1553            51   
           North Side                     238      2093            56   
           Northwest Side                 211      2103            71   
           South Side                     333      2505            99   
           Southwest Side                 453      3759           121   
           West Side                      782      5480           206   

LIGHTING_GRP                             Dawn                          \
INJURY_GRP                    Moderate injury No injury Severe injury   
CRASH_YEAR SIDE                                                         
2018       Central                         18       206             5   
           Far North Side                  21       240             3   
           Far Southeast Side              14       166             6   
           Far Southwest Side              17       112  

## Type

In [4]:
crash_type = df.pivot_table(values='CRASH_RECORD_ID', index=['CRASH_YEAR', 'SIDE'], columns=['FIRST_CRASH_TYPE'], aggfunc='count', fill_value=0)
crash_type.head()

FIRST_CRASH_TYPE               ANGLE  ANIMAL  FIXED OBJECT  HEAD ON  \
CRASH_YEAR SIDE                                                       
2018       Central               838       2           528       60   
           Far North Side       1608      29           582      122   
           Far Southeast Side   1117      12           635       85   
           Far Southwest Side    841       3           360       57   
           North Side            968       4           398       97   

FIRST_CRASH_TYPE               OTHER NONCOLLISION  OTHER OBJECT  OVERTURNED  \
CRASH_YEAR SIDE                                                               
2018       Central                             46            91           2   
           Far North Side                      47           132           4   
           Far Southeast Side                  53           126          15   
           Far Southwest Side                  14            73           7   
           North Side                          41            91           1   

FIRST_CRASH_TYPE               PARKED MOTOR VEHICLE  PEDALCYCLIST  PEDESTRIAN  \
CRASH_YEAR SIDE                                                                 
2018       Central                             1638           264         413   
           Far North Side                      3512           276         375   
           Far Southeast Side                  2151            43         209   
           Far Southwest Side                  1693            48         171   
           North Side                          2448           365         342   

FIRST_CRASH_TYPE               REAR END  REAR TO FRONT  REAR TO REAR  \
CRASH_YEAR SIDE                                                        
2018       Central                 2854              0             0   
           Far North Side          3886              0             0   
           Far Southeast Side      1988              0             0   
           Far Southwest Side      1814              0             0   
           North Side              2775              0             0   

FIRST_CRASH_TYPE               REAR TO SIDE  SIDESWIPE OPPOSITE DIRECTION  \
CRASH_YEAR SIDE                                                             
2018       Central                        0                           142   
           Far North Side                 0                           263   
           Far Southeast Side             0                           131   
           Far Southwest Side             0                            88   
           North Side                     0                           197   

FIRST_CRASH_TYPE               SIDESWIPE SAME DIRECTION  TRAIN  TURNING  
CRASH_YEAR SIDE                                                          
2018       Central                                 3633      0     2010  
           Far North Side                          2279      0     1994  
           Far Southeast Side                      1134      1     1224  
           Far Southwest Side                      1006      0     1137  
           North Side                              1753      0     1512